# `objectives`

talk about obj vs constraints
- [ ] why do we need to re-make the objectives when we change eq resolutoin
  - I think this is because the objectives get built for that eq (built = ?) , so remaking them means we get rid of the built status?
- is the size of the `x_reduced` equal to number of parameters? YES IT IS And is this equal to one of the sides of the linear constraint matrix `A`? it is in `factorize_linear_constraints` which is in `objectives.utils`, from `project` function

## `linear_objectives.py`

- when specifying interior surface as the fixboundary constraint, the self A becomes zernike_radial instead of 1?

## `objectives/utils.py`

#### `factorize_linear_constraints`

 - define problem in standard optimization setup
 - match which DESC variables are equal to the standard ones
 - write Ax=b
 - this thing basically finds the nullspace( A )=: Z
  - Z then can be multiplied with (x - x_particular) to obtain a vector which is guaranteed to be in the nullspace of the constraints, bc it is made up of a linear combinations of vectors which lie inside of the nullspace (does this make sense?)...

DESC approaches the ideal MHD fixed-boundary equilibrium problem $\mathbf{F}=0$ [as an optimization problem](https://desc-docs.readthedocs.io/en/latest/theory_general.html):

$$\begin{align}
\min_{\mathbf{x}\in \mathcal{R}^n} \mathbf{f}(\mathbf{x})&\\
\text{subject to the linear constraints}~~ \mathbf{A}\mathbf{x}=\mathbf{b}&
\end{align}$$

where the objective to be minimized is the MHD force balance error $\mathbf{F}=\mathbf{J}\times \mathbf{B} - \nabla p$, which is minimized by evaluating  the two components of $\mathbf{F}$ on a collocation grid (resulting in a vector of residuals $\mathbf{f} = [f_{\rho},f_{\beta}]$ of length `2*num_nodes` since each of $f_{\rho},f_{\beta}$ are evaluated at the collocation nodes) and then minimizing those residuals $\mathbf{f}(\mathbf{x})$. 
The state variable being minimized over  $\mathbf{x} = [R_{lmn}, Z_{lmn}, \lambda_{lmn}]$ is the vector of the Fourier-Zernike spectral coefficients used to describe the mapping between the toroidal $(R,\phi,Z)$ coordinates and the computational flux coordinates $(\rho,\theta,\zeta)$.
The state is of length `3*eq.R_basis.num_modes` (if a non-stellarator symmetric equilibrium, where the number of basis modes for R and Z are the same), or length `eq.R_basis.num_modes + 2 * eq.Z_basis.num_modes` (if a stellarator-symmetric equilibrium, where $R$ has $cos(m\theta-n\zeta)$ symmetry and $Z$ and $\lambda$ have $sin(m\theta-n\zeta)$).


A fixed-boundary equilbrium problem requires the fixed-boundary $R_b(\theta,\zeta),Z_b{\theta,\zeta}$ to be given as the input, and this appears as a linear constraint on $\mathbf{x}$ during the optimization. 
In DESC, additionally a [gauge constraint](https://desc-docs.readthedocs.io/en/latest/_api/objectives/desc.objectives.FixLambdaGauge.html) on $\lambda$ is applied, since $\lambda$ is only defined up to an additive multiple of $2\pi$, which constitutes another linear constraint to the problem. 
The linear constraints are then written in the form $\mathbf{A}\mathbf{x}=\mathbf{b}$. 

In solving for the equilibrium DESC deals with these linear constraints by using the feasible direction formulation (see for example [page 3 of this reference](https://www.cs.umd.edu/users/oleary/a607/607constr1hand.pdf)).

The state variable $\mathbf{x}$ is written as $\mathbf{x} = \mathbf{x}_p + \mathbf{Z}\mathbf{y}$